#### 두개의 투자전략을 클래스로 선언
1. 생성자 함수
  - 클래스가 생성될 때 최초로 한번만 실행이 되는 함수
  - 객체 변수(self 변수)의 데이터프레임과, 기준이 되는 컬럼명, 시작시간, 종료시간
2. buyandhold 함수
  - 생성자 함수에서 만든 객체 변수를 이용하여 백테스팅을 돌림
  - 결과와 누적 수익률을 리턴
3. bollinger 함수
  - 밴드 생성 함수
    - 상단밴드, 하단밴드, 이동 평균선 생성
  - 거래 내역 추가 함수
    - 밴드를 기준으로 거래 내역을 생성
  - 수익률 계산 함수
    - 매도 시 수익률 발생하고 데이터프레임과 누적 수익률을 리턴

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_5100\3713710582.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [42]:
class Invest:
    # 생성자 함수
    def __init__(self, 
                 _df, 
                 _col = 'Adj Close', 
                 _start = "2010-01-01", 
                 _end = datetime.now()):
        # _df이 결측치와 무한대 값 제외시키기
        flag = _df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        self.df = _df.loc[~flag,]
        # 데이터프레임에서 컬럼에 Date 가 포함되어 있는가?
        if 'Date' in self.df.columns:
            self.df.set_index('Date', inplace=True)
        # 인덱스를 시계열데이터로 변경
        self.df.index = pd.to_datetime(self.df.index, format='%Y-%m-%d')
        self.df = self.df[[_col]]
        self.col = _col
        try:
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            if type(_end) == 'str':
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else:
                self.end = _end
        except:
            print('투자기간의 인자값의 포맷이 잘못되엇습니다.(yyyy-mm-dd)')
    ## buyandhold 함수 생성
    def buyandhold(self):
        # 투자기간으로 데이터를 필터링
        result = self.df.loc[self.start : self.end, ]
        # 일일 수익률 컬럼을 생성
        result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)
        # 누적 수익률 컬럼을 생성
        result['acc_rtn'] = result['rtn'].cumprod()
        # 총 누적수익률 데이터만 추출
        acc_rtn = result.iloc[-1,]['acc_rtn']
        return result, acc_rtn
    
    ## bollinger 함수 생성
    # 밴드 생성 함수 생성
    def bollinger(self, _cnt = 20):
        result = self.df.copy()
        # 이동 평균선 생성
        result['center'] = result[self.col].rolling(_cnt).mean()
        # 상단 밴드를 생성
        result['ub']= result['center']+(2*result[self.col].rolling(_cnt).std())
        # 하단 밴드를 생성
        result['lb']= result['center']-(2*result[self.col].rolling(_cnt).std())
        # 투자기간으로 데이터를 필터
        result = result.loc[self.start : self.end,]
        # 거래 내역 추가
        result['trade'] = ""
        for i in result.index:
            # 상단밴드보다 기준이 되는 컬럼의 데이터가 크거나 같은 경우
            if result.loc[i, self.col] >= result.loc[i, 'ub']:
                # 현재 보유중이라면 매도, 보유중 아니라면 유지
                result.loc[i, 'trade']= ""
            # 하단밴드보다 기준이 되는 컬럼의 데이터가 작거나 같은 경우
            elif result.loc[i, self.col] <= result.loc[i, 'lb']:
                # 현재 보유중이 아니라면 매수, 보유중이면 유지
                result.loc[i, 'trade'] = 'buy'
            # 밴드 사이에 기준되는 컬럼의 데이터가 존재한다면
            else:
                # 현재 보유중이라면 보유, 보유중이 아니라면 보유하지 않음
                if (result.shift().loc[i, 'trade'] == ""):
                    result.loc[i, 'trade'] = result.shift().loc[i, 'trade']
        
        # 수익률 계산
        result['rtn'] = 1

        for i in result.index:
            # 매수의 경우(전날의 trade가 비어있고 오늘의 trade가 buy인 경우)
            if (result.shift().loc[i, 'trade'] == "") & (result.loc[i,'trade'] == 'buy'):
                # 구매가 대입
                buy = result.loc[i, self.col]
                print(f"매수일 : {i}, 매수가 : {buy} ")
            # 매도의 경우(전날의 trade가 buy, 오늘의 trade가 "" 인 경우)
            elif (result.shift().loc[i, 'trade'] == 'buy') & (result.loc[i, 'trade'] == ""):
                # 판매가 대입
                sell = result.loc[i, self.col]
                # 수익률 계산
                rtn = sell/buy
                # 수익률을 판매한 날의 rtn 컬럼에 대입
                result.loc[i, 'rtn'] = rtn
                print(f"매도일 : {i}, 매도가 : {sell}, 수익률 : {round(rtn, 2)}")

        # 누적수익률 계산
        result['acc_rtn'] = result['rtn'].cumprod()
        acc_rtn = result.iloc[-1]['acc_rtn']        
        return result, acc_rtn
    
        


In [43]:
AAPL = pd.read_csv('C:/Users/SAMSUNG/Documents/R/csv/AAPL.csv')

In [44]:
class1 = Invest(AAPL)

In [22]:
bnh_df, bnh_rtn = class1.buyandhold()

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_5100\2850493333.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)
C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_5100\2850493333.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['acc_rtn'] = result['rtn'].cumprod()


In [23]:
bnh_rtn

7.436513727083075

In [24]:
bnh_df.tail()

,Adj Close,rtn,acc_rtn
Date,,,
2019-06-18,198.449997,1.023518,7.409631
2019-06-19,197.869995,0.997077,7.387975
2019-06-20,199.460007,1.008036,7.447342
2019-06-21,198.779999,0.996591,7.421952
2019-06-24,199.169998,1.001962,7.436514


In [41]:
class1.bollinger()['trade'].value_counts()

trade
       1426
buy     945
Name: count, dtype: int64

In [40]:
class1.bollinger(10)

,Adj Close,center,ub,lb,trade
Date,,,,,
2010-01-04,26.782711,25.808441,27.450318,24.166563,None
2010-01-05,26.829010,26.045594,27.493890,24.597298,None
2010-01-06,26.402260,26.205031,27.371338,25.038724,None
2010-01-07,26.353460,26.332932,27.186903,25.478960,None
2010-01-08,26.528664,26.456577,26.900384,26.012771,None
...,...,...,...,...,...
2019-06-18,198.449997,191.871998,201.349270,182.394727,
2019-06-19,197.869995,193.404999,200.932158,185.877840,
2019-06-20,199.460007,194.828999,200.674109,188.983890,


In [45]:
class1.bollinger()

매수일 : 2010-01-22 00:00:00, 매수가 : 24.747818 
매도일 : 2010-01-25 00:00:00, 매도가 : 25.413599, 수익률 : 1.03
매수일 : 2010-01-28 00:00:00, 매수가 : 24.940546 
매도일 : 2010-02-01 00:00:00, 매도가 : 24.369871, 수익률 : 0.98
매수일 : 2010-08-24 00:00:00, 매수가 : 30.026524 
매도일 : 2010-08-25 00:00:00, 매도가 : 30.396961, 수익률 : 1.01
매수일 : 2011-03-16 00:00:00, 매수가 : 41.299767 
매도일 : 2011-03-17 00:00:00, 매도가 : 41.879189, 수익률 : 1.01
매수일 : 2011-05-16 00:00:00, 매수가 : 41.711502 
매도일 : 2011-05-17 00:00:00, 매도가 : 42.066914, 수익률 : 1.01
매수일 : 2011-06-10 00:00:00, 매수가 : 40.785408 
매도일 : 2011-06-13 00:00:00, 매도가 : 40.873005, 수익률 : 1.0
매수일 : 2011-06-20 00:00:00, 매수가 : 39.461357 
매도일 : 2011-06-21 00:00:00, 매도가 : 40.710316, 수익률 : 1.03


C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_5100\3569613967.py:83: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0269026142021895' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[i, 'rtn'] = rtn


매수일 : 2011-11-14 00:00:00, 매수가 : 47.463268 
매도일 : 2011-11-15 00:00:00, 매도가 : 48.660904, 수익률 : 1.03
매수일 : 2011-11-21 00:00:00, 매수가 : 46.180508 
매도일 : 2011-11-22 00:00:00, 매도가 : 47.119102, 수익률 : 1.02
매수일 : 2012-04-16 00:00:00, 매수가 : 72.601524 
매도일 : 2012-04-17 00:00:00, 매도가 : 76.302109, 수익률 : 1.05
매수일 : 2012-04-20 00:00:00, 매수가 : 71.706749 
매도일 : 2012-04-23 00:00:00, 매도가 : 71.546547, 수익률 : 1.0
매수일 : 2012-04-24 00:00:00, 매수가 : 70.117378 
매도일 : 2012-04-25 00:00:00, 매도가 : 76.339684, 수익률 : 1.09
매수일 : 2012-05-17 00:00:00, 매수가 : 66.342926 
매도일 : 2012-05-18 00:00:00, 매도가 : 66.375458, 수익률 : 1.0
매수일 : 2012-10-08 00:00:00, 매수가 : 80.207954 
매도일 : 2012-10-09 00:00:00, 매도가 : 79.916374, 수익률 : 1.0
매수일 : 2012-10-19 00:00:00, 매수가 : 76.647339 
매도일 : 2012-10-22 00:00:00, 매도가 : 79.68763, 수익률 : 1.04
매수일 : 2012-11-02 00:00:00, 매수가 : 72.494728 
매도일 : 2012-11-05 00:00:00, 매도가 : 73.477585, 수익률 : 1.01
매수일 : 2012-11-07 00:00:00, 매수가 : 70.452194 
매도일 : 2012-11-09 00:00:00, 매도가 : 69.0709, 수익률 : 0.98
매수일 : 2013-01-15

(             Adj Close      center          ub          lb trade  rtn  acc_rtn
 Date                                                                          
 2010-01-04   26.782711   25.037723   27.046734   23.028713        1.0  1.00000
 2010-01-05   26.829010   25.169503   27.288098   23.050908        1.0  1.00000
 2010-01-06   26.402260   25.307290   27.366449   23.248130        1.0  1.00000
 2010-01-07   26.353460   25.436879   27.410937   23.462821        1.0  1.00000
 2010-01-08   26.528664   25.525609   27.529742   23.521475        1.0  1.00000
 ...                ...         ...         ...         ...   ...  ...      ...
 2019-06-18  198.449997  185.432500  201.032574  169.832427        1.0  1.42265
 2019-06-19  197.869995  185.996000  202.558154  169.433846        1.0  1.42265
 2019-06-20  199.460007  186.830000  204.361771  169.298229        1.0  1.42265
 2019-06-21  198.779999  187.786000  205.751400  169.820600        1.0  1.42265
 2019-06-24  199.169998  188.796000  206